# Memory

> Semantic memory capabilities for Cogitarelink DSPy agents

This module provides memory capabilities for Cogitarelink DSPy agents, enabling them to store and retrieve reflections as JSON-LD entities. The memory system allows agents to persist "lessons learned" and later recall this information to improve their capabilities over time.

## Overview

The memory system implements three key features:

1. **Persistence**: Store agent reflections as semantic JSON-LD entities with proper provenance
2. **Retrieval**: Retrieve the most relevant reflections based on recency and tags
3. **Prompt Injection**: Format reflections for inclusion in system prompts

This implementation integrates with the Cogitarelink graph system and follows a semantic approach where reflections are formalized as entities with types, references, and timestamps.

## Architecture

The memory system uses these key components:

- **ReflectionStore**: The main class for storing and retrieving reflections
- **ReflectionNote**: The entity type (defined in the vocabulary registry)
- **DSPy Tool Integration**: Tools for DSPy agents to interact with reflections

The tools are exposed through the components registry as:
- `AddReflection`: Store new reflections
- `RecallReflection`: Retrieve stored reflections
- `ReflectionPrompt`: Format reflections for prompt injection

## Evaluation

The memory system includes a dedicated evaluation harness in `tests/devset_memory.jsonl` that tests:
- Storing new reflections
- Retrieving specific knowledge
- Formatting for prompt inclusion

These tools are optimized using DSPy's compilation framework for optimal performance.

In [ ]:
#| default_exp memory

In [ ]:
#| export
import datetime, uuid, dspy
from typing import List, Optional
from cogitarelink.core.graph import GraphManager
from cogitarelink.core.entity import Entity
from cogitarelink.reason.prov import wrap_patch_with_prov

REFLECTION_GRAPH = "urn:agent:reflections"
REFLECTION_TYPE  = "https://w3id.org/cogitarelink#ReflectionNote"

class ReflectionStore:
    """Persist 'lesson learned' notes as JSON-LD entities in the Cogitarelink graph.
    
    This class provides a semantic memory store for agent reflections, storing them
    as properly typed JSON-LD entities in the Cogitarelink graph with full provenance.
    It enables storing, retrieving, and formatting reflections for use in prompts.
    
    Attributes:
        graph (GraphManager): The Cogitarelink graph manager instance to use for storage
    """
    def __init__(self, graph: GraphManager):
        """Initialize the reflection store with a graph manager.
        
        Args:
            graph (GraphManager): The Cogitarelink graph manager for accessing the knowledge graph
        """
        self.graph = graph

    def add(self, text: str, tags: Optional[List[str]] = None) -> str:
        """Add a new reflection note to the semantic memory.
        
        This method creates a new ReflectionNote entity with the given text and tags,
        assigns it a UUID, timestamps it, and stores it in the graph with provenance.
        
        Args:
            text (str): The reflection text to store
            tags (List[str], optional): A list of tags to categorize this reflection
            
        Returns:
            str: The ID of the newly created reflection
        """
        note_id = f"urn:uuid:{uuid.uuid4()}"
        now     = datetime.datetime.utcnow().isoformat()
        content = {
            "@id": note_id,
            "@type": REFLECTION_TYPE,
            "text": text,
            "tags": tags or [],
            "dateCreated": now
        }
        ent = Entity(vocab=["clref","schema"], content=content)
        with wrap_patch_with_prov(
            self.graph, source="urn:agent:self",
            agent="urn:agent:self", activity="urn:agent:addReflection"
        ):
            self.graph.ingest_entity(ent)
        return note_id

    def retrieve(self, limit: int = 5, tag_filter: Optional[str] = None) -> List[Entity]:
        """Fetch up to `limit` most recent notes, optionally filtering by tag.
        
        This method retrieves reflection notes from the graph, sorted by creation date
        (newest first), and optionally filtered by a specific tag.
        
        Args:
            limit (int, optional): Maximum number of notes to retrieve. Defaults to 5.
            tag_filter (str, optional): Only return notes with this tag. Defaults to None.
            
        Returns:
            List[Entity]: List of reflection notes as Entity objects
        """
        ids = []
        triples = self.graph.query(
            pred="http://schema.org/dateCreated",
            graph_id=REFLECTION_GRAPH
        )
        triples.sort(key=lambda t: t[2], reverse=True)
        for s,_,_ in triples[:limit]:
            if tag_filter:
                tag_triples = self.graph.query(
                    subj=s, pred="http://schema.org/tags", graph_id=REFLECTION_GRAPH
                )
                tags = [o for (_,_,o) in tag_triples]
                if tag_filter not in tags:
                    continue
            ids.append(s)
        ents = []
        for nid in ids:
            t = self.graph.query(
                subj=nid, pred="http://schema.org/text", graph_id=REFLECTION_GRAPH
            )
            text = t[0][2] if t else ""
            ents.append(Entity(vocab=["clref","schema"], content={
                "@id": nid,
                "@type": REFLECTION_TYPE,
                "text": text
            }))
        return ents

    def as_prompt(self, limit: int = 5) -> str:
        """Format retrieved reflections for inclusion in a system prompt.
        
        This method retrieves recent reflections and formats them as a 
        bulleted list suitable for inclusion in system prompts.
        
        Args:
            limit (int, optional): Maximum number of notes to include. Defaults to 5.
            
        Returns:
            str: Formatted string with bullet points for each reflection
        """
        notes = self.retrieve(limit)
        return "\n".join(f"• {e.content['text']}" for e in notes)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()